# 5. Gelişmiş Model Analizi ve İyileştirmeler

Bu notebook'ta önerilen geliştirmeleri uygulayacağız:

## İçerik:
1. Log transform ile fiyat dönüşümü
2. Gelişmiş feature engineering
3. Gruplandırılmış cross-validation
4. Mantıksal veri filtreleri
5. Kategori embedding ve target encoding
6. Model güven aralıkları
7. Etik ve yasal konular


In [ ]:
# Gerekli kütüphaneleri import edelim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Makine öğrenmesi
from sklearn.model_selection import train_test_split, KFold, GroupKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Açıklanabilir AI
import shap
from lime.lime_tabular import LimeTabularExplainer

# Diğer
import joblib
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

print("Kütüphaneler yüklendi!")


In [ ]:
# Gerçek Kaggle veri setini yükleyelim
df = pd.read_csv('../data/kaggle_arac_verisi_processed.csv')

print("Veri seti yüklendi!")
print(f"Boyut: {df.shape}")
print(f"Sütunlar: {list(df.columns)}")

# İlk birkaç satırı görelim
df.head()


In [ ]:
# Veri seti hakkında detaylı bilgi
print("=== VERİ SETİ BİLGİLERİ ===")
print(f"Toplam kayıt sayısı: {len(df):,}")
print(f"Toplam sütun sayısı: {df.shape[1]}")
print("\n=== EKSİK DEĞER ANALİZİ ===")
missing_data = df.isnull().sum()
if missing_data.sum() > 0:
    print(missing_data[missing_data > 0])
else:
    print("Eksik değer bulunamadı!")

print("\n=== FİYAT ANALİZİ ===")
print(f"Min fiyat: {df['fiyat'].min():,.0f} TL")
print(f"Max fiyat: {df['fiyat'].max():,.0f} TL")
print(f"Ortalama fiyat: {df['fiyat'].mean():,.0f} TL")
print(f"Medyan fiyat: {df['fiyat'].median():,.0f} TL")
print(f"Standart sapma: {df['fiyat'].std():,.0f} TL")

# Fiyat dağılımını görselleştirelim
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Ham fiyat dağılımı
df['fiyat'].hist(bins=100, ax=axes[0], alpha=0.7)
axes[0].set_title('Ham Fiyat Dağılımı')
axes[0].set_xlabel('Fiyat (TL)')
axes[0].set_ylabel('Frekans')

# Log transform uygulanmış fiyat
df['log_fiyat'] = np.log1p(df['fiyat'])
df['log_fiyat'].hist(bins=100, ax=axes[1], alpha=0.7, color='orange')
axes[1].set_title('Log Transform Uygulanmış Fiyat Dağılımı')
axes[1].set_xlabel('Log(Fiyat + 1)')
axes[1].set_ylabel('Frekans')

plt.tight_layout()
plt.show()

print(f"\nLog transform sonrası:")
print(f"Min log fiyat: {df['log_fiyat'].min():.2f}")
print(f"Max log fiyat: {df['log_fiyat'].max():.2f}")
print(f"Ortalama log fiyat: {df['log_fiyat'].mean():.2f}")


In [ ]:
# Mantıksal veri filtreleri - veri kalitesini artıralım
print("=== VERİ TEMİZLEME ===")

# Önceki durumu kaydet
print(f"Filtreleme öncesi kayıt sayısı: {len(df):,}")

# Mantıksal filtreler
df_clean = df.copy()

# 1. Kilometre filtreleri
df_clean = df_clean[(df_clean['kilometre'] >= 0) & (df_clean['kilometre'] <= 1_000_000)]
print(f"Kilometre filtresi sonrası: {len(df_clean):,}")

# 2. Model yılı filtreleri
df_clean = df_clean[(df_clean['model_yili'] >= 1980) & (df_clean['model_yili'] <= 2025)]
print(f"Model yılı filtresi sonrası: {len(df_clean):,}")

# 3. Motor hacmi filtreleri
df_clean = df_clean[(df_clean['motor_hacmi'] >= 500) & (df_clean['motor_hacmi'] <= 8000)]
print(f"Motor hacmi filtresi sonrası: {len(df_clean):,}")

# 4. Fiyat filtreleri (çok düşük ve çok yüksek fiyatları temizle)
Q1_price = df_clean['fiyat'].quantile(0.01)  # %1'lik dilim
Q99_price = df_clean['fiyat'].quantile(0.99)  # %99'luk dilim
df_clean = df_clean[(df_clean['fiyat'] >= Q1_price) & (df_clean['fiyat'] <= Q99_price)]
print(f"Fiyat filtresi sonrası: {len(df_clean):,}")

# 5. Motor gücü filtreleri (varsa)
if 'motorgucu' in df_clean.columns:
    df_clean = df_clean[(df_clean['motorgucu'] >= 30) & (df_clean['motorgucu'] <= 1000)]
    print(f"Motor gücü filtresi sonrası: {len(df_clean):,}")

print(f"\nToplam {len(df) - len(df_clean):,} kayıt temizlendi")
print(f"Kalan kayıt sayısı: {len(df_clean):,}")

# Temizlenmiş veriyi kullan
df = df_clean.copy()


In [ ]:
# Gelişmiş Feature Engineering
print("=== FEATURE ENGINEERING ===")

# 1. Araç yaşı
df['arac_yasi'] = 2025 - df['model_yili']

# 2. Yıllık ortalama kilometre
df['yillik_ortalama_km'] = df['kilometre'] / (df['arac_yasi'] + 1)  # +1 ile sıfıra bölmeyi önle

# 3. KM per motor hacmi
df['km_per_motor_hacmi'] = df['kilometre'] / (df['motor_hacmi'] + 0.1)

# 4. Motor gücü per hacim (varsa)
if 'motorgucu' in df.columns:
    df['guc_per_hacim'] = df['motorgucu'] / (df['motor_hacmi'] + 0.1)

# 5. Fiyat per yaş (değer kaybı oranı)
df['fiyat_per_yas'] = df['fiyat'] / (df['arac_yasi'] + 1)

# 6. Yaş grubu kategorileri
df['yas_grubu'] = pd.cut(df['arac_yasi'], 
                        bins=[0, 3, 7, 15, 100], 
                        labels=['Yeni (0-3)', 'Az Kullanılmış (3-7)', 'Orta (7-15)', 'Eski (15+)'])

# 7. Kilometre grubu kategorileri
df['km_grubu'] = pd.cut(df['kilometre'], 
                       bins=[0, 50000, 100000, 200000, 1000000], 
                       labels=['Düşük (0-50k)', 'Orta (50k-100k)', 'Yüksek (100k-200k)', 'Çok Yüksek (200k+)'])

# 8. Motor hacmi grubu
df['motor_grubu'] = pd.cut(df['motor_hacmi'], 
                          bins=[0, 1200, 1600, 2000, 8000], 
                          labels=['Küçük (0-1.2L)', 'Orta (1.2-1.6L)', 'Büyük (1.6-2.0L)', 'Çok Büyük (2.0L+)'])

print("Yeni özellikler oluşturuldu:")
new_features = ['arac_yasi', 'yillik_ortalama_km', 'km_per_motor_hacmi', 'yas_grubu', 'km_grubu', 'motor_grubu']
for feature in new_features:
    if feature in df.columns:
        print(f"  ✓ {feature}")

# Kategorik değişkenleri sayısal değerlere çevir (Label Encoding)
categorical_features = ['marka', 'seri', 'model', 'vites_turu', 'yakit_turu', 'kasatipi', 'cekis', 'yas_grubu', 'km_grubu', 'motor_grubu']

label_encoders = {}
for feature in categorical_features:
    if feature in df.columns:
        le = LabelEncoder()
        df[feature + '_encoded'] = le.fit_transform(df[feature].astype(str))
        label_encoders[feature] = le

print(f"\nToplam sütun sayısı: {df.shape[1]}")
print(f"Yeni özellik sayısı: {len(new_features) + len(categorical_features)}")
